<a href="https://colab.research.google.com/github/ShrutiKharate/KNN-Classification-and-KD-Tree-Optimization/blob/main/KD_Tree_Nearest_Neighbor_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##🌳 Part B — KD-Tree Nearest-Neighbor Queries Solution

In [1]:
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KDTree
from scipy.stats import mode # Used for finding the majority label

# --- Assumed Setup from Part A (Run this if starting fresh) ---
# Load and clean data
path = kagglehub.dataset_download("gregorut/videogamesales")
df = pd.read_csv(f"{path}/vgsales.csv")
cols_to_check = ['Year', 'Genre', 'Platform', 'Global_Sales']
df_clean = df.dropna(subset=cols_to_check).copy()
df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce').astype('Int64').clip(lower=1980, upper=2020)

# Define features and target (as used for the classifier)
numeric_features = ['Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
categorical_features = ['Platform']
X_raw = df_clean[numeric_features + categorical_features]
y_genre = df_clean['Genre']

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Stratified Train/Test Split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_genre, test_size=0.3, random_state=42, stratify=y_genre
)

# Apply preprocessing and get the scaled training and test features
X_train_scaled = preprocessor.fit_transform(X_train_raw)
X_test_scaled = preprocessor.transform(X_test_raw)

# Get the original index for easy lookup later
train_indices = X_train_raw.index
test_indices = X_test_raw.index

print("Data preparation complete. Features are scaled and split.")

100%|██████████| 381k/381k [00:00<00:00, 54.9MB/s]

Extracting files...


Data preparation complete. Features are scaled and split.


In [2]:
## Task: Build a KD-tree on the scaled training features
k_neighbors = 5 # Number of neighbors to query

# The KDTree requires a dense numpy array
kdtree = KDTree(X_train_scaled.toarray())
print(f"\nKD-Tree successfully built on {X_train_scaled.shape[0]} training samples.")

# Select three random indices from the TEST set for query
np.random.seed(42)
query_indices_in_test_set = np.random.choice(X_test_scaled.shape[0], 3, replace=False)

print(f"Querying for k={k_neighbors} nearest neighbors for 3 test samples...")

results = {}

for i, query_idx_in_test in enumerate(query_indices_in_test_set):
    query_point = X_test_scaled[query_idx_in_test].toarray()

    # Query the KD-Tree
    distances, neighbor_indices_in_train = kdtree.query(query_point, k=k_neighbors)

    # Get the true genre of the test point
    true_label = y_test.iloc[query_idx_in_test]

    # Map the neighbor indices back to the original index and Genre label
    original_neighbor_indices = train_indices[neighbor_indices_in_train[0]]
    neighbor_labels = y_train.loc[original_neighbor_indices].values

    # Store results for analysis
    results[i] = {
        'Test Index (in test set)': query_idx_in_test,
        'True Label': true_label,
        'Distances': distances[0],
        'Neighbor Indices (in train set)': original_neighbor_indices,
        'Neighbor Labels': neighbor_labels
    }

    # Print required output
    print(f"\n--- Query Sample {i+1} ---")
    print(f"True Genre of Test Point: **{true_label}**")
    print(f"Distances to {k_neighbors} Neighbors: {np.round(distances[0], 4)}")
    print(f"Neighbor Indices (Original Index): {original_neighbor_indices.tolist()}")
    print(f"Neighbor Genre Labels: {neighbor_labels}")


KD-Tree successfully built on 11428 training samples.
Querying for k=5 nearest neighbors for 3 test samples...

--- Query Sample 1 ---
True Genre of Test Point: **Action**
Distances to 5 Neighbors: [0.     0.     0.     0.0122 0.024 ]
Neighbor Indices (Original Index): [13508, 13170, 13029, 12523, 13095]
Neighbor Genre Labels: ['Racing' 'Sports' 'Racing' 'Shooter' 'Role-Playing']

--- Query Sample 2 ---
True Genre of Test Point: **Action**
Distances to 5 Neighbors: [0.253  0.2619 0.2934 0.3076 0.3086]
Neighbor Indices (Original Index): [2594, 2381, 2589, 2045, 2620]
Neighbor Genre Labels: ['Action' 'Shooter' 'Misc' 'Action' 'Role-Playing']

--- Query Sample 3 ---
True Genre of Test Point: **Action**
Distances to 5 Neighbors: [0.032  0.032  0.0531 0.0531 0.0552]
Neighbor Indices (Original Index): [4947, 4987, 4418, 4920, 4726]
Neighbor Genre Labels: ['Adventure' 'Simulation' 'Puzzle' 'Racing' 'Action']


In [4]:
## For one example, compare the majority label of neighbors with the model prediction and the true label.

# Use the results from the first query sample (i=0)
comparison_sample = results[0]

true_label = comparison_sample['True Label']
neighbor_labels = comparison_sample['Neighbor Labels']

# Find the majority (predicted) label using np.unique
unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
majority_index = np.argmax(counts)
majority_label = unique_labels[majority_index]
majority_count = counts[majority_index]


print("\n--- Majority Label Comparison ---")
print(f"Sample Used (Index in Test Set): {comparison_sample['Test Index (in test set)']}")
print(f"True Genre Label: **{true_label}**")
print(f"Neighbor Labels: {neighbor_labels}")
print(f"Majority (Predicted) Genre Label (from {k_neighbors} neighbors): **{majority_label}**")
print(f"Count of Majority Label: {majority_count}/{k_neighbors}")

# Comparison
if true_label == majority_label:
    print("\n✅ **Comparison Result:** The **majority label matches the true label**. This means the KNN model (with k=5) would correctly classify this test point.")
else:
    print(f"\n❌ **Comparison Result:** The majority label **({majority_label}) does not match the true label ({true_label})**. The KNN model (with k=5) would misclassify this test point.")


--- Majority Label Comparison ---
Sample Used (Index in Test Set): 4656
True Genre Label: **Action**
Neighbor Labels: ['Racing' 'Sports' 'Racing' 'Shooter' 'Role-Playing']
Majority (Predicted) Genre Label (from 5 neighbors): **Racing**
Count of Majority Label: 2/5

❌ **Comparison Result:** The majority label **(Racing) does not match the true label (Action)**. The KNN model (with k=5) would misclassify this test point.
